![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**0- Connect to your Google Drive folder**

In [ ]:
my_folder = "/content/drive/MyDrive/COLAB-SAS"

from google.colab import drive
drive.mount('/content/drive')

# Change the following code to set your Drive folder
import os
os.chdir(my_folder)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/COLAB-SAS


**1- Make sure that your Python version is 3.3 or higher as well as your Java version is 1.8.0_162 or higher**

In [ ]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [ ]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [ ]:
pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 12.0 MB/s 
  Created wheel for saspy: filename=saspy-4.4.0-py3-none-any.whl size=9937189 sha256=124b8ac4fb98dfbd28654b51e1117a11034603826d6dfec6dfd6497bae389270
  Stored in directory: /root/.cache/pip/wheels/78/ce/27/57cfb223c6e6232856fe149e532b99faeaf94b8d47bc273ccb
Successfully built saspy


**3- Create the configuration file "sascfg_personal.py"**
Please, check that your Home Region is correct, you can check it at [ODA-SAS](https://welcome.oda.sas.com/home)

In [ ]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 1
'iomhost' : ['odaws01-usw2.oda.sas.com','odaws02-usw2.oda.sas.com','odaws03-usw2.oda.sas.com','odaws04-usw2.oda.sas.com'],
#US Home Region 2
#'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
#European Home Region 1
#'iomhost' : ['odaws01-euw1.oda.sas.com','odaws02-euw1.oda.sas.com'],
#Asia Pacific Home Region 1
#'iomhost' : ['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'],
#Asia Pacific Home Region 2
#'iomhost' : ['odaws01-apse1-2.oda.sas.com','odaws02-apse1-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create your .authinfo**

If there is no .authinfo file, you can create this

In [ ]:
#%%writefile .authinfo
#oda user USR password PSW

Copy this file to home

In [ ]:
!cp .authinfo ~/.authinfo

cp: cannot stat '.authinfo': No such file or directory


**5- Establish Connection (Need to do this step each time you use SASPy)**

In [ ]:
import saspy
sas_session = saspy.SASsession(cfgfile=os.path.join(
    my_folder,"sascfg_personal.py"))
sas_session

Using SAS Config named: oda
Error trying to read authinfo file:/root/.authinfo
[Errno 2] No such file or directory: '/root/.authinfo'
Did not find key oda in authinfo file:/root/.authinfo

Please enter the OMR user id: A00827714@tec.mx
Please enter the password for OMR user : ··········
SAS Connection established. Subprocess id is 380



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/COLAB-SAS/sascfg_personal.py
WORK Path             = /saswork/SAS_work340E0001E6A9_odaws02-usw2.oda.sas.com/SAS_workCA970001E6A9_odaws02-usw2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.0
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 124585


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [ ]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                    Thursday, December  1, 2022 09:17:00 AM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                    Thursday, December  1, 2022 09:17:00 AM

33         


For local Jupyter

In [ ]:
#%%SAS sas_session
#libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [ ]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
171103,2171104.0,BLAKELY,REFUGIO,1.0,53.0,47800.0,University / College,Rural,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,20.0
237951,2237952.0,FERRELL,STACEY,1.0,76.0,54800.0,University / College,Rural,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,20.0


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [ ]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['Referrals']]=data_merge[['Referrals']]/data_merge[['Referrals']].max()
data_merge[['Recency']]=data_merge[['Recency']]/data_merge[['Recency']].max()
data_merge[['Frequency']]=data_merge[['Frequency']]/data_merge[['Frequency']].max()
data_merge[['MinGift']]=data_merge[['MinGift']]/data_merge[['MinGift']].max()
data_merge[['MaxGift']]=data_merge[['MaxGift']]/data_merge[['MaxGift']].max()
data_merge[['TotalGift']]=data_merge[['TotalGift']]/data_merge[['TotalGift']].max()
data_merge[['Age']]=data_merge[['Age']]/data_merge[['Age']].max()
data_merge[['Salary']]=data_merge[['Salary']]/data_merge[['Salary']].max()
data_merge[['AmtLastYear']]=data_merge[['AmtLastYear']]/data_merge[['AmtLastYear']].max()
data_merge[['NbActivities']]=data_merge[['NbActivities']]/data_merge[['NbActivities']].max()



data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=min('MinGift'))  
data_merge[['Referrals']] = data_merge[['Referrals']].fillna(value=0)  
data_merge[['Recency']] = data_merge[['Recency']].fillna(value=0)  
data_merge[['Frequency']] = data_merge[['Frequency']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['TotalGift']].fillna(value=0)
data_merge[['Seniority']] = data_merge[['Seniority']].fillna(value=0)


data_merge.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
822441,2822442.0,SPENCER,CYNTHIA,1.0,0.233333,0.0544,University / College,Suburban,0.0,0.0,...,0.0,0.0,0.000000,0.0000,0.0000,1.0,0.015,0.0,1.0,20.0
888644,2888645.0,CRUZ,BRIAN,0.0,0.466667,0.0316,High School,Suburban,1.0,0.0,...,0.0,0.0,0.000000,0.0000,0.0000,0.0,0.000,0.0,1.0,10.0
369046,2369047.0,BEALS,AMY,1.0,0.188889,0.0528,High School,Suburban,7.0,0.4,...,0.3,6.0,0.006168,0.0025,0.0025,0.0,0.000,1.0,1.0,20.0
126208,2126209.0,MAHLER,JENNY,1.0,0.433333,0.0232,High School,Suburban,0.0,0.0,...,0.0,0.0,0.000000,0.0000,0.0000,0.0,0.000,1.0,1.0,20.0
24986,2024987.0,GRUBER,KAREN,1.0,0.455556,0.0984,High School,Suburban,4.0,0.0,...,0.0,0.0,0.000000,0.0000,0.0000,0.0,0.000,0.0,1.0,20.0


## Data Partition

In [ ]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.6,random_state=5678) # you can change the percentage
train.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
779084,2779085.0,GARRINGER,URSULA,1.0,0.711111,0.0596,University / College,Rural,0.0,0.0,...,0.0,0.0,0.000000,0.000,0.000,0.0,0.000,0.0,1.0,50.0
468414,2468415.0,BELIVEAU,JACK,0.0,0.733333,0.0888,High School,City,10.0,0.0,...,0.1,6.0,0.008224,0.010,0.010,0.0,0.000,0.0,1.0,20.0
884277,2884278.0,SMITH,KIMBERLY,1.0,0.544444,0.3004,High School,City,0.0,0.0,...,0.0,0.0,0.000000,0.000,0.000,0.0,0.000,0.0,1.0,30.0
292121,2292122.0,KING,SONIA,1.0,0.222222,0.0100,High School,Suburban,7.0,0.1,...,0.1,6.0,0.002467,0.003,0.003,1.0,0.002,0.0,1.0,20.0
286180,2286181.0,ROMAN,TERRY,1.0,0.622222,0.5344,University / College,Rural,8.0,0.0,...,0.0,0.0,0.000000,0.000,0.000,0.0,0.000,1.0,1.0,75.0



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [ ]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[ 55.18768883  10.08880502 120.48555583 ...  57.76864062 103.61834029
  94.87321866]


In [ ]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

63.71826959746686
55042.725192508515
234.61186072427907


## Regression Tree Model（Py）

In [ ]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','MinGift','Contact', 'AmtLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','MinGift','Contact', 'AmtLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[ 52.75721154  28.52422003 156.55990017 ...  49.71985384 156.55990017
  65.456358  ]


In [ ]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

62.959705805619606
55112.95173726083
234.76147839298685


In [ ]:
from sklearn.neural_network import MLPRegressor

X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 
Y_valid = validation['AmtThisYear']

regr = MLPRegressor(max_iter=1000).fit(X_train,Y_train)
MLPregr_predict=regr.predict(X_valid)

In [ ]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,MLPregr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,MLPregr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,MLPregr_predict)))

64.29943691664937
54680.50513954409
233.83863055437203


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


## Scoring New Data

### Prepare data for scoring

In [ ]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [ ]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_contact[['MinGift']] = scoring_data_contact[['MinGift']].fillna(value=0) 
scoring_data_contact[['Referrals']] = scoring_data_contact[['Referrals']].fillna(value=0)  
scoring_data_contact[['Frequency']] = scoring_data_contact[['Frequency']].fillna(value=0)
scoring_data_contact[['MaxGift']] = scoring_data_contact[['MaxGift']].fillna(value=0)
scoring_data_contact[['TotalGift']] = scoring_data_contact[['TotalGift']].fillna(value=0)
scoring_data_contact[['Seniority']] = scoring_data_contact[['Seniority']].fillna(value=0)
scoring_data_contact[['Recency']] = scoring_data_contact[['Recency']].fillna(value=0)




X = scoring_data_contact[['Age', 'Salary','Contact','MinGift', 'AmtLastYear','GaveLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 

regr_predict_contact=regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,ID,AmtContact
0,2000001.0,6.841346e+06
1,2000002.0,9.776406e+05
2,2000003.0,3.556186e+06
3,2000004.0,1.465356e+06
4,2000005.0,4.455097e+06


### Predict 'amount given' for members who were not contacted

In [ ]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_nocontact[['MinGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)
scoring_data_nocontact[['Referrals']] = scoring_data_nocontact[['Referrals']].fillna(value=0)  
scoring_data_nocontact[['Frequency']] = scoring_data_nocontact[['Frequency']].fillna(value=0)
scoring_data_nocontact[['MaxGift']] = scoring_data_nocontact[['MaxGift']].fillna(value=0)
scoring_data_nocontact[['TotalGift']] = scoring_data_nocontact[['TotalGift']].fillna(value=0)
scoring_data_nocontact[['Seniority']] = scoring_data_nocontact[['Seniority']].fillna(value=0)
scoring_data_nocontact[['Recency']] = scoring_data_nocontact[['Recency']].fillna(value=0)


#scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age', 'Salary','MinGift','Contact', 'AmtLastYear','GaveLastYear','Woman', 'NbActivities','Referrals','Recency','Frequency','MaxGift','TotalGift','SeniorList']] 

regr_predict_nocontact=regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,ID,AmtNoContact
0,2000001.0,6.839159e+06
1,2000002.0,9.776461e+05
2,2000003.0,3.556192e+06
3,2000004.0,1.465361e+06
4,2000005.0,4.455103e+06


In [ ]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,6.841346e+06,6.839159e+06
1,2000002.0,9.776406e+05,9.776461e+05
2,2000003.0,3.556186e+06,3.556192e+06


## Exporting Results to a CSV File

In [ ]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.

In [ ]:
!head Round2_Output_amt.csv

ID,AmtContact,AmtNoContact
2000001.0,6841345.627479889,6839158.649791403
2000002.0,977640.5763559225,977646.105054292
2000003.0,3556186.226381161,3556191.75507953
2000004.0,1465355.7751471186,1465361.303845488
2000005.0,4455097.393768585,4455102.922466955
2000006.0,256567.13168447232,256572.66038284183
2000007.0,141417.9269784106,141423.4556767801
2000008.0,5744542.876984574,5744548.405682944
2000009.0,1380612.6487721896,1380618.177470559
